In [10]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta
from sqlalchemy.orm import sessionmaker
import os, sys

project_dir = os.path.abspath('..')
sys.path.insert(0, project_dir)

from sqlalchemy import create_engine
import pandas as pd
import pandas_market_calendars as mcal

import unittest
from unittest.mock import patch
from volatility_forecast.data.dataloader import (
    TiingoEoDDataLoader,
    PriceVolumeDatabaseLoader,
    TiingoEodDataLoaderProd,
)

from volatility_forecast.data.date_util import get_closest_next_business_day, get_closest_prev_business_day
from volatility_forecast.data.base import Field, DataSet
from volatility_forecast.data.dataset import PriceVolume
from volatility_forecast.data.persistence import persist_data, load_data_from_db
from volatility_forecast.data.database import engine, Base


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:


class TestLoaderAndPersistence():

    def __init__(self):
        # Create tables
        Base.metadata.create_all(engine)

        self.tickers = (
            "AAPL",
            "GOOGL",
            "MSFT",
        )
        self.end_date = get_closest_prev_business_day(
            datetime.now().date()  - timedelta(days=5),
            mcal.get_calendar("NYSE"),
        )
        self.start_date = get_closest_next_business_day(
            self.end_date - timedelta(days=30),
            mcal.get_calendar("NYSE"),
        )
        self.loader = TiingoEodDataLoaderProd(self.tickers)

    def setUp(self):
        # This method will be called before each test
        pass

    def tearDown(self):
        # This method will be called after each test
        pass

    def test_fetch_and_persist_data(self):
        data = self.loader.get_data(
            fields=[PriceVolume.CLOSE, PriceVolume.VOLUME],
            start_date=self.start_date,
            end_date=self.end_date,
        )

        self.assertIsNotNone(data)
        self.assertEqual(len(data), 2)  # CLOSE and VOLUME
        self.assertIn(PriceVolume.CLOSE, data)
        self.assertIn(PriceVolume.VOLUME, data)

        for field, field_data in data.items():
            self.assertIsInstance(field_data, pd.DataFrame)
            self.assertEqual(len(field_data.columns), len(self.tickers))

    def test_data_in_database(self):
        for ticker in self.tickers:
            db_data = load_data_from_db(ticker, self.start_date, self.end_date)
            self.assertIsNotNone(db_data)
            self.assertFalse(db_data.empty)
            self.assertTrue("adjClose" in db_data.columns)
            self.assertTrue("adjVolume" in db_data.columns)

    def test_fetch_from_database(self):
        # Fetch data twice
        data1 = self.loader.get_data(
            fields=[PriceVolume.CLOSE, PriceVolume.VOLUME],
            start_date=self.start_date,
            end_date=self.end_date,
        )

        data2 = self.loader.get_data(
            fields=[PriceVolume.CLOSE, PriceVolume.VOLUME],
            start_date=self.start_date,
            end_date=self.end_date,
        )

        # Verify that the data is the same
        for field in [PriceVolume.CLOSE, PriceVolume.VOLUME]:
            pd.testing.assert_frame_equal(data1[field], data2[field])

    def test_fetch_new_data(self):
        new_end_date = get_closest_next_business_day(
            self.end_date + timedelta(days=1),
            mcal.get_calendar("NYSE"),
        )
        new_data = self.loader.get_data(
            fields=[PriceVolume.CLOSE, PriceVolume.VOLUME],
            start_date=self.start_date,
            end_date=new_end_date,
        )

        for field, field_data in new_data.items():
            self.assertTrue(field_data.index.max().date() >= self.end_date)


In [12]:
TestLoaderAndPersistence().test_fetch_new_data()

AttributeError: 'TestLoaderAndPersistence' object has no attribute 'assertTrue'

In [ ]:
os.getcwd()

'/Users/steveyang/Projects/Github/volatility-forecast/notebook'